In [1]:
import numpy as np
import pandas as pd
import json

In [2]:
with open ('/content/intents.json') as file:
  data=json.load(file)

In [3]:
training_sentence=[]
training_labels=[]
labels=[]
response=[]

In [4]:
for intent in data['intents']:
  for pattern in intent['patterns']:
    training_sentence.append(pattern)
    training_labels.append(intent['tag'])
  if intent['tag'] not in labels:
    labels.append(intent['tag'])

num_classes=len(labels)

In [5]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

In [6]:
tokenizer=Tokenizer()
tokenizer.fit_on_texts(training_sentence)
sentence=tokenizer.texts_to_sequences(training_sentence)
padded_sequence=pad_sequences(sentence,padding='pre',maxlen=36)

In [7]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [8]:
le=LabelEncoder()
le.fit(training_labels)
y=le.transform(training_labels)

In [9]:
import keras
from keras.layers import Dense,GlobalAveragePooling1D,Embedding
from keras.models import Sequential

In [10]:
len(list(set(training_sentence)))

39

In [11]:
max([len(s)for s in training_sentence])

36

In [12]:
model=Sequential()
model.add(Embedding(input_dim=1000,output_dim=20,input_length=36))
model.add(GlobalAveragePooling1D())
model.add(Dense(16,activation='relu'))
model.add(Dense(16,activation='relu'))
model.add(Dense(num_classes,activation='softmax'))
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics='accuracy')
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 36, 20)            20000     
_________________________________________________________________
global_average_pooling1d (Gl (None, 20)                0         
_________________________________________________________________
dense (Dense)                (None, 16)                336       
_________________________________________________________________
dense_1 (Dense)              (None, 16)                272       
_________________________________________________________________
dense_2 (Dense)              (None, 10)                170       
Total params: 20,778
Trainable params: 20,778
Non-trainable params: 0
_________________________________________________________________


In [13]:
model.fit(padded_sequence,np.array(y),epochs=600)

Epoch 1/600
2/2 [==============================] - 1s 9ms/step - loss: 2.3023 - accuracy: 0.1250
Epoch 2/600
2/2 [==============================] - 0s 5ms/step - loss: 2.3010 - accuracy: 0.1250
Epoch 3/600
2/2 [==============================] - 0s 8ms/step - loss: 2.3005 - accuracy: 0.1250
Epoch 4/600
2/2 [==============================] - 0s 6ms/step - loss: 2.2993 - accuracy: 0.1250
Epoch 5/600
2/2 [==============================] - 0s 6ms/step - loss: 2.2986 - accuracy: 0.1250
Epoch 6/600
2/2 [==============================] - 0s 11ms/step - loss: 2.2973 - accuracy: 0.1250
Epoch 7/600
2/2 [==============================] - 0s 6ms/step - loss: 2.2964 - accuracy: 0.1250
Epoch 8/600
2/2 [==============================] - 0s 4ms/step - loss: 2.2942 - accuracy: 0.1354
Epoch 9/600
2/2 [==============================] - 0s 4ms/step - loss: 2.2949 - accuracy: 0.1146
Epoch 10/600
2/2 [==============================] - 0s 12ms/step - loss: 2.2934 - accuracy: 0.1250
Epoch 11/600
2/2 [=========

In [14]:
model.save('chat_bot')

INFO:tensorflow:Assets written to: chat_bot/assets


In [15]:
import pickle

In [16]:
with open('tokenizer.pickle','wb') as tok:
  pickle.dump(tokenizer,tok,protocol=pickle.HIGHEST_PROTOCOL)

In [17]:
with open('labelencoder.pickle','wb') as labelencoder:
  pickle.dump(le,labelencoder,protocol=pickle.HIGHEST_PROTOCOL)

In [18]:
def chat():
  model=keras.models.load_model('chat_bot')
  with open('tokenizer.pickle','rb') as tok:
    tokenizer=pickle.load(tok)

  with open('labelencoder.pickle','rb') as lel:
    labele=pickle.load(lel)

  while True:
    inp=input()
    if inp.lower()=='quit':
      break
    result=model.predict(pad_sequences(tokenizer.texts_to_sequences([inp]),padding='pre',maxlen=36))
    tags=labele.inverse_transform([np.argmax(result)])
    for i in data['intents']:
      if i['tag']==tags:
        print('chatbot',np.random.choice(i['responses']))

print('Hello I Am ChatBot-')
chat()

Hello I Am ChatBot-
hi
chatbot Hi
who are you
chatbot I.m Joana, your bot assistant
what do you sell
chatbot we sell mogo oil at 10$ for one liter
where is your office
chatbot we are in goa please share your contact no. so we can contact you
I want to raise a complaint
chatbot Please mention your complaint, we will reach you and sorry for any inconvenience caused
quit
